In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score
from scipy.optimize import curve_fit
from uncertainties import ufloat

In [14]:
### Plotting raw data only for copper, axis = 0 ###
rawdata = pd.read_csv("C:\\Users\\chril\\Documents\\Python Scripts\\Fluid\\ENGR120-Lab-Report-6\\Python\\Lab6\\Copper\\copper_fl_u.csv", delimiter=",", header=None)
rawdata = rawdata[rawdata.iloc[:,2] == 0]
rawdata.iloc[:,0] = rawdata.iloc[:,0].replace("2024-11-12_","", regex=True)
rawdata.iloc[:,0] = pd.to_datetime(rawdata.iloc[:,0], format="%H:%M:%S.%f")

plt.plot(rawdata.iloc[:,0], rawdata.iloc[:,3], '.', label="Raw data")
plt.xlabel("Time")
plt.ylabel("Acceleration")
plt.savefig("C:\\Users\\chril\\Documents\\Python Scripts\\Fluid\\ENGR120-Lab-Report-6\\Python\\Lab6\\Plots\\rawdataCUa0.png")
plt.close()

In [15]:
def fitfunc(x, a, b):
    return a*x + b

In [92]:
def R2Score(Material):
    FullData = pd.DataFrame()
    filename = "C:\\Users\\chril\\Documents\\Python Scripts\\Fluid\\ENGR120-Lab-Report-6\\Python\\Lab6\\" + Material.capitalize() + "\\" + Material + "_fl_"
    Df = pd.DataFrame()
    for i in ["02", "03", "04", "06", "u"]:
        Data = pd.read_csv(filename + i + ".csv", delimiter = ",", header = None)
        Df[Material + i + "_time"] = Data[0] #Names the columns
        Df[Material + i + "_axis"] = Data[2]
        Df[Material + i + "_acceleration"] =Data[3]
    Df = Df.dropna() #Makes sure they have run for equal amount of time

    results = pd.DataFrame()
    for axis in range(3): #Calculates the R2 score for each axis
        AvgAcc, stdAcc, FlowRate = [], [], []
        AxisDf = Df[Df[Material + "02_axis"]  == axis] #New temporary dataframe for each axis

        for i in ["02", "03", "04", "06"]: #Calculates the average acceleration for each flowrate
            AvgAcc.append(AxisDf[Material + i + "_acceleration"].mean()) 
            stdAcc.append(AxisDf[Material + i + "_acceleration"].std()/np.sqrt(len(AxisDf[Material + i + "_acceleration"])))
            FlowRate.append(int(i))

        par, cov = curve_fit(fitfunc, FlowRate, AvgAcc, sigma = stdAcc, absolute_sigma=True) #Fits the data to a linear function

        unkownAcc = AxisDf[Material + "u_acceleration"].mean() #Calculates the unknown flowrate
        
        dict = {
            "Material": [Material], 
            "Axis": [axis],  
            "R2_score": [r2_score(AvgAcc, fitfunc(np.asarray(FlowRate), *par))], 
            "Unknown flowrate": [(unkownAcc - par[1])/par[0] ],
            "Flowrate errror": [np.sqrt((1/par[0])**2* (np.std(AxisDf[Material + "u_acceleration"])/len(AxisDf[Material + "u_acceleration"]))**2
                                        + (1/par[0])**2*cov[1,1] 
                                        + ((unkownAcc - par[1])/(par[0]**2))**2*cov[0,0])],
            } #Creates a dictionary with the results
        results = pd.DataFrame(dict) #Creates a dataframe with the results
        #write to csv
        results.to_csv("C:\\Users\\chril\\Documents\\Python Scripts\\Fluid\\ENGR120-Lab-Report-6\\Python\\Lab6\\Results\\" + Material + str(axis)+ "_R2_score.csv", header = True)
        
        if axis == 0: #Plots the data for all materials, only for axis = 0
            a = ufloat(par[0], np.sqrt(cov[0,0])) #Calculates the parameters and uncertainties
            b = ufloat(par[1], np.sqrt(cov[1,1]))
            fig = plt.figure()
            plt.errorbar(FlowRate, AvgAcc, yerr = stdAcc, fmt = "o", label = "Data")
            plt.plot(
                FlowRate, 
                fitfunc(np.asarray(FlowRate), *par), 
                label = f"Fit, $A = {a:L}, B = {b:L}$"
                )
            plt.xlabel("Flowrate")
            plt.ylabel("Acceleration")
            plt.legend()
            plt.savefig("C:\\Users\\chril\\Documents\\Python Scripts\\Fluid\\ENGR120-Lab-Report-6\\Python\\Lab6\\Plots\\" + Material + "_axis" + str(axis) + ".png", bbox_inches='tight')
            plt.close(fig)


In [93]:
for mat in ["copper", "galvanized", "pvc"]:
    R2Score(mat)
# plt.close()

In [94]:
Fulldata = pd.DataFrame()
for mat in ["copper", "galvanized", "pvc"]:
    for axis in range(3):
        data = pd.read_csv("C:\\Users\\chril\\Documents\\Python Scripts\\Fluid\\ENGR120-Lab-Report-6\\Python\\Lab6\\Results\\" + mat + str(axis) + "_R2_score.csv", delimiter = ",", header = 0)
        data["Unknown flowrate"] = "${:L}$".format(ufloat(data["Unknown flowrate"].values[0], data["Flowrate errror"].values[0]))
        #Add data to FullData
        Fulldata = pd.concat([Fulldata, data], ignore_index = True)
        # print(data["Material"].values[0], 
        #       round(data["Axis"].values[0], 2), 
        #       round(data["R2_score"].values[0], 2), 
                # "${:L}$".format(ufloat(data["Unknown flowrate"].values[0], data["Flowrate errror"].values[0])).replace(" ", "")
        # )
# columnnames = ["Material", "Axis", "$R^2 score$", "Unknown flowrate"]
# columnnames = ["\textbf{" + i + "}" for i in columnnames]
# print(columnnames)
Fulldata.rename(columns = {"Material": "\\textbf{Material}", "Axis": "\\textbf{Axis}", "R2_score": "\\textbf{$R^2$ score}", "Unknown flowrate": "\\textbf{Unknown flowrate $[m^3 / hr]$}"}, inplace = True)
LatexTable = Fulldata.drop(columns = ["Unnamed: 0", "Flowrate errror"]).to_latex(index = False, float_format="%.2f")
#Savve asd as .tex
with open("C:\\Users\\chril\\Documents\\Python Scripts\\Fluid\\ENGR120-Lab-Report-6\\Python\\Lab6\\Results\\R2_score.tex", "w") as text_file:
    text_file.write(LatexTable)
print(LatexTable)

\begin{tabular}{lrrl}
\toprule
\textbf{Material} & \textbf{Axis} & \textbf{$R^2$ score} & \textbf{Unknown flowrate $[m^3 / hr]$} \\
\midrule
copper & 0 & 0.73 & $5.3 \pm 3.3$ \\
copper & 1 & 0.38 & $6.0 \pm 1.8$ \\
copper & 2 & 0.50 & $0 \pm 8$ \\
galvanized & 0 & 0.64 & $4.3 \pm 2.9$ \\
galvanized & 1 & 0.27 & $3 \pm 4$ \\
galvanized & 2 & 0.39 & $10 \pm 9$ \\
pvc & 0 & 0.90 & $6.2 \pm 1.0$ \\
pvc & 1 & 0.07 & $-7 \pm 11$ \\
pvc & 2 & -0.37 & $1 \pm 5$ \\
\bottomrule
\end{tabular}



In [ ]:
#Converts the notebook to a markdown
!jupyter nbconvert --to markdown --output-dir="C:/Users/chril/Documents/Python Scripts/Fluid/ENGR120-Lab-Report-6" Lab6.ipynb


[NbConvertApp] Converting notebook Lab6.ipynb to markdown
[NbConvertApp] Writing 4972 bytes to C:\Users\chril\Documents\Python Scripts\Fluid\ENGR120-Lab-Report-6\Lab6.md
